In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<font size="6", color="red">ch3. 연관분석</font>
- pip install apyori
# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석
- 활용분야 : 이벤트 미리감지(사기적발...), 신상품카테고리 구성

[조건:left-hand side:오렌지주스] -> [결과:right-hand side:와인]
- 연관분석과 관련된 지표
```
1. 지지도(support) : 얼마나 자주 함께 나타나는지
    (lhs, rhs)의 항목수/전체항목수 = 0.2
2. 신뢰도(confidence) : 조건이 오면 결과가 얼마나 자주 나타나는지
    (lhs->rhs)의 항목수/ lhs의 항목수 = 1/2 = 0.5
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인
    lhs->rhs의 지지도 / ( lhs의 항목수 * rhs의 항목수 )
    => 0.2 / (0.4*0.6) = 0.2/0.24 = 0.833
    
    향상도 < 1 : 기대가 낮다
    
    향상도 > 1 : 기대가 높다
    
```

# 2. 연관분석 구현

In [2]:
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    #print(list(csvdata))
    transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [3]:
from apyori import apriori
rules = apriori(transaction, #2차원 데이터
               min_supoort=0.15,
               min_confidence=0.1)
rules = list(rules)

In [4]:
rules[10]

RelationRecord(items=frozenset({'콜라', '소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '소주'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

In [5]:
rule = rules[10]
support = rule[1]
order_st = rule[2]
for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence = item[2]
    lift = item[3]
    if lift > 1:
        print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                             round(confidence,2), 
                                             round(lift,2)))

frozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
frozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25


In [6]:
rule = rules[10]
support = rule[1]
order_st = rule[2]
for item in order_st:
    lhs = item[0]
    rhs = item[1]
    confidence=item[2]
    lift = item[3]
    if lift > 1 :
        print("P{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, round(confidence,2), round(lift,2)))

Pfrozenset({'소주'})=>frozenset({'콜라'})	 0.6	 1.0	 1.25
Pfrozenset({'콜라'})=>frozenset({'소주'})	 0.6	 0.75	 1.25


In [7]:
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                                 round(confidence,2), 
                                                 round(lift,2)))

맥주=>콜라	 0.4	 1.0	 1.25
콜라=>맥주	 0.4	 0.5	 1.25
소주=>콜라	 0.6	 1.0	 1.25
콜라=>소주	 0.6	 0.75	 1.25
콜라=>맥주, 소주	 0.2	 0.25	 1.25
맥주, 소주=>콜라	 0.2	 1.0	 1.25
맥주=>와인, 콜라	 0.2	 0.5	 1.25
콜라=>와인, 맥주	 0.2	 0.25	 1.25
와인, 맥주=>콜라	 0.2	 1.0	 1.25
와인, 콜라=>맥주	 0.2	 0.5	 1.25
소주=>콜라, 오렌지주스	 0.2	 0.33	 1.67
콜라=>오렌지주스, 소주	 0.2	 0.25	 1.25
오렌지주스, 소주=>콜라	 0.2	 1.0	 1.25
콜라, 오렌지주스=>소주	 0.2	 1.0	 1.67
콜라=>와인, 소주	 0.2	 0.25	 1.25
와인, 소주=>콜라	 0.2	 1.0	 1.25


In [8]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
# rules_df.loc[0] = [ '와인', '오렌지', 0.15, 0.5, 1.1] 식으로 for문 내에서 데이터 추가
idx = 0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift,2)]
            idx += 1
            print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                                 round(confidence,2), 
                                                 round(lift,2)))
rules_df.sort_values(by=['향상도', '신뢰도'], ascending=False)

맥주=>콜라	 0.4	 1.0	 1.25
콜라=>맥주	 0.4	 0.5	 1.25
소주=>콜라	 0.6	 1.0	 1.25
콜라=>소주	 0.6	 0.75	 1.25
콜라=>맥주, 소주	 0.2	 0.25	 1.25
맥주, 소주=>콜라	 0.2	 1.0	 1.25
맥주=>와인, 콜라	 0.2	 0.5	 1.25
콜라=>와인, 맥주	 0.2	 0.25	 1.25
와인, 맥주=>콜라	 0.2	 1.0	 1.25
와인, 콜라=>맥주	 0.2	 0.5	 1.25
소주=>콜라, 오렌지주스	 0.2	 0.33	 1.67
콜라=>오렌지주스, 소주	 0.2	 0.25	 1.25
오렌지주스, 소주=>콜라	 0.2	 1.0	 1.25
콜라, 오렌지주스=>소주	 0.2	 1.0	 1.67
콜라=>와인, 소주	 0.2	 0.25	 1.25
와인, 소주=>콜라	 0.2	 1.0	 1.25


,lhs,rhs,지지도,신뢰도,향상도
13,"콜라, 오렌지주스",소주,0.2,1.00,1.67
10,소주,"콜라, 오렌지주스",0.2,0.33,1.67
0,맥주,콜라,0.4,1.00,1.25
2,소주,콜라,0.6,1.00,1.25
5,"맥주, 소주",콜라,0.2,1.00,1.25
8,"와인, 맥주",콜라,0.2,1.00,1.25
12,"오렌지주스, 소주",콜라,0.2,1.00,1.25
15,"와인, 소주",콜라,0.2,1.00,1.25
3,콜라,소주,0.6,0.75,1.25
1,콜라,맥주,0.4,0.50,1.25


# 3. 경주/전주 여행 자료 연관분석

In [22]:
import pandas as pd
from konlpy.tag import Hannanum, Kkma, Komoran
from apyori import apriori
df = pd.read_csv('data/naver_kin.csv', sep='\t')
total_text_list = list(df['total_text'])
# total_text_list[:2]
# analyzer = Hannanum()
analyzer = Komoran()
total_noun_list = []
# select_pos = ['NC', 'NQ'] # Hannanum 보통명사, 고유명사
select_pos = ['NNG', 'NNP'] # Kkma, Komoran 보통명사, 고유명사
불용어 = {'여행', '전주여행', '경주여행'}
for total_text in total_text_list :
#     total_noun = analyzer.nouns(total_text) # 의존명사, 수사 포함됨
    total_noun = [ token for token, tag in analyzer.pos(total_text)
                     if (tag in select_pos) & (token not in 불용어)& (len(token)>1)]
    total_noun_list.append(total_noun)
print(total_noun_list[:2])

[['전주', '한곳', '추천', '추억', '체험', '감성', '추억', '테마', '박물관', '시간', '전북', '전북', '투어', '패스', '통합', '이용', '여행지', '체험', '카페', '이용', '추가', '전주', '필수', '편안', '감사'], ['전주', '전주', '사람과', '호텔', '가격', '얼마', '정도', '맛집', '카페', '추천', '안녕하세요', '전주', '계획', '한옥마을', '근처']]


In [23]:
%%time 
rules = apriori(total_noun_list, min_support=0.15, min_confidence=0.2) # 데이터 부족하면 에러남
rules = list(rules)
len(rules)

CPU times: total: 93.8 ms
Wall time: 77.3 ms


194

In [24]:
rules_df = pd.DataFrame(None, columns=['lhs','rhs','지지도','신뢰도','향상도'])
idx = 0
for rule in rules:
    support = rule[1]
    order_st = rule[2]
    for item in order_st:
        lhs = ', '.join([data for data in item[0]])
        rhs = ', '.join([data for data in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            rules_df.loc[idx] = [lhs, rhs, support, round(confidence, 2), round(lift,2)]
            idx += 1
            print("{}=>{}\t {}\t {}\t {}".format(lhs, rhs, support, 
                                                 round(confidence,2), 
                                                 round(lift,2)))
rules_df.sort_values(by=['향상도', '신뢰도'], ascending=False, inplace=True)
rules_df = rules_df.reset_index(drop=True)

경주=>추천	 0.221	 0.44	 1.02
추천=>경주	 0.221	 0.51	 1.02
국내=>맛집	 0.167	 0.97	 3.16
맛집=>국내	 0.167	 0.54	 3.16
국내=>여수	 0.157	 0.91	 5.31
여수=>국내	 0.157	 0.91	 5.31
국내=>전주	 0.167	 0.97	 1.94
전주=>국내	 0.167	 0.33	 1.94
국내=>주가	 0.157	 0.91	 5.31
주가=>국내	 0.157	 0.91	 5.31
국내=>코스	 0.162	 0.94	 1.84
코스=>국내	 0.162	 0.32	 1.84
국내=>한곳	 0.157	 0.91	 3.93
한곳=>국내	 0.157	 0.68	 3.93
맛집=>여수	 0.167	 0.54	 3.16
여수=>맛집	 0.167	 0.97	 3.16
맛집=>전주	 0.252	 0.82	 1.64
전주=>맛집	 0.252	 0.5	 1.64
맛집=>제주도	 0.152	 0.5	 3.26
제주도=>맛집	 0.152	 1.0	 3.26
맛집=>주가	 0.172	 0.56	 3.26
주가=>맛집	 0.172	 1.0	 3.26
맛집=>코스	 0.217	 0.71	 1.38
코스=>맛집	 0.217	 0.42	 1.38
맛집=>한곳	 0.187	 0.61	 2.63
한곳=>맛집	 0.187	 0.81	 2.63
여수=>전주	 0.172	 1.0	 2.0
전주=>여수	 0.172	 0.34	 2.0
여수=>제주도	 0.152	 0.88	 5.81
제주도=>여수	 0.152	 1.0	 5.81
여수=>주가	 0.167	 0.97	 5.64
주가=>여수	 0.167	 0.97	 5.64
여수=>코스	 0.167	 0.97	 1.9
코스=>여수	 0.167	 0.33	 1.9
여수=>한곳	 0.167	 0.97	 4.19
한곳=>여수	 0.167	 0.72	 4.19
전주=>제주도	 0.152	 0.3	 2.0
제주도=>전주	 0.152	 1.0	 2.0
전주=>주가	 0.172	 0.34	

여수, 국내=>전주, 맛집	 0.157	 1.0	 3.97
전주, 국내=>맛집, 여수	 0.157	 0.94	 5.63
맛집, 여수=>전주, 국내	 0.157	 0.94	 5.63
전주, 맛집=>여수, 국내	 0.157	 0.62	 3.97
전주, 여수=>맛집, 국내	 0.157	 0.91	 5.47
맛집, 여수, 국내=>전주	 0.157	 1.0	 2.0
전주, 맛집, 국내=>여수	 0.157	 0.94	 5.47
전주, 여수, 국내=>맛집	 0.157	 1.0	 3.26
전주, 맛집, 여수=>국내	 0.157	 0.94	 5.47
국내=>맛집, 여수, 주가	 0.157	 0.91	 5.47
맛집=>국내, 여수, 주가	 0.157	 0.51	 3.26
여수=>맛집, 주가, 국내	 0.157	 0.91	 5.81
주가=>맛집, 여수, 국내	 0.157	 0.91	 5.81
맛집, 국내=>여수, 주가	 0.157	 0.94	 5.63
여수, 국내=>맛집, 주가	 0.157	 1.0	 5.81
주가, 국내=>맛집, 여수	 0.157	 1.0	 5.99
맛집, 여수=>국내, 주가	 0.157	 0.94	 5.99
맛집, 주가=>여수, 국내	 0.157	 0.91	 5.81
여수, 주가=>맛집, 국내	 0.157	 0.94	 5.63
맛집, 여수, 국내=>주가	 0.157	 1.0	 5.81
맛집, 주가, 국내=>여수	 0.157	 1.0	 5.81
주가, 여수, 국내=>맛집	 0.157	 1.0	 3.26
맛집, 여수, 주가=>국내	 0.157	 0.94	 5.47
국내=>맛집, 코스, 여수	 0.157	 0.91	 5.47
맛집=>코스, 여수, 국내	 0.157	 0.51	 3.26
여수=>맛집, 코스, 국내	 0.157	 0.91	 5.63
코스=>맛집, 여수, 국내	 0.157	 0.31	 1.96
맛집, 국내=>코스, 여수	 0.157	 0.94	 5.63
여수, 국내=>맛집, 코스	 0.157	 1.0	 4.61
코스, 국내=>맛집, 여수	 0.157	 0

제주도, 주가=>맛집, 여수	 0.152	 1.0	 5.99
맛집, 여수, 제주도=>주가	 0.152	 1.0	 5.81
맛집, 여수, 주가=>제주도	 0.152	 0.91	 5.99
맛집, 주가, 제주도=>여수	 0.152	 1.0	 5.81
제주도, 여수, 주가=>맛집	 0.152	 1.0	 3.26
맛집=>코스, 여수, 제주도	 0.152	 0.5	 3.26
여수=>맛집, 코스, 제주도	 0.152	 0.88	 5.81
제주도=>맛집, 코스, 여수	 0.152	 1.0	 5.99
코스=>맛집, 여수, 제주도	 0.152	 0.3	 1.96
맛집, 여수=>코스, 제주도	 0.152	 0.91	 5.99
맛집, 제주도=>코스, 여수	 0.152	 1.0	 5.99
코스, 맛집=>제주도, 여수	 0.152	 0.7	 4.61
제주도, 여수=>맛집, 코스	 0.152	 1.0	 4.61
코스, 여수=>맛집, 제주도	 0.152	 0.91	 5.99
코스, 제주도=>맛집, 여수	 0.152	 1.0	 5.99
맛집, 여수, 제주도=>코스	 0.152	 1.0	 1.96
코스, 맛집, 여수=>제주도	 0.152	 0.91	 5.99
코스, 맛집, 제주도=>여수	 0.152	 1.0	 5.81
코스, 제주도, 여수=>맛집	 0.152	 1.0	 3.26
맛집=>제주도, 여수, 한곳	 0.152	 0.5	 3.26
여수=>제주도, 맛집, 한곳	 0.152	 0.88	 5.81
제주도=>맛집, 여수, 한곳	 0.152	 1.0	 5.99
한곳=>맛집, 여수, 제주도	 0.152	 0.66	 4.31
맛집, 여수=>제주도, 한곳	 0.152	 0.91	 5.99
맛집, 제주도=>여수, 한곳	 0.152	 1.0	 5.99
맛집, 한곳=>제주도, 여수	 0.152	 0.81	 5.35
제주도, 여수=>맛집, 한곳	 0.152	 1.0	 5.35
여수, 한곳=>맛집, 제주도	 0.152	 0.91	 5.99
제주도, 한곳=>맛집, 여수	 0.152	 1.0	 5.99
맛집, 

한곳, 여수, 주가=>제주도	 0.152	 0.91	 5.99
한곳, 제주도, 주가=>여수	 0.152	 1.0	 5.81
여수=>한곳, 코스, 제주도	 0.152	 0.88	 5.81
제주도=>코스, 여수, 한곳	 0.152	 1.0	 5.99
코스=>제주도, 여수, 한곳	 0.152	 0.3	 1.96
한곳=>코스, 여수, 제주도	 0.152	 0.66	 4.31
제주도, 여수=>코스, 한곳	 0.152	 1.0	 5.38
코스, 여수=>제주도, 한곳	 0.152	 0.91	 5.99
여수, 한곳=>코스, 제주도	 0.152	 0.91	 5.99
코스, 제주도=>여수, 한곳	 0.152	 1.0	 5.99
제주도, 한곳=>코스, 여수	 0.152	 1.0	 5.99
코스, 한곳=>제주도, 여수	 0.152	 0.82	 5.38
코스, 제주도, 여수=>한곳	 0.152	 1.0	 4.31
제주도, 여수, 한곳=>코스	 0.152	 1.0	 1.96
코스, 여수, 한곳=>제주도	 0.152	 0.91	 5.99
코스, 제주도, 한곳=>여수	 0.152	 1.0	 5.81
여수=>코스, 주가, 한곳	 0.167	 0.97	 5.64
주가=>코스, 여수, 한곳	 0.167	 0.97	 5.81
코스=>주가, 여수, 한곳	 0.167	 0.33	 1.96
한곳=>코스, 여수, 주가	 0.167	 0.72	 4.31
여수, 주가=>코스, 한곳	 0.167	 1.0	 5.38
코스, 여수=>주가, 한곳	 0.167	 1.0	 5.81
여수, 한곳=>코스, 주가	 0.167	 1.0	 5.81
코스, 주가=>여수, 한곳	 0.167	 0.97	 5.81
한곳, 주가=>코스, 여수	 0.167	 0.97	 5.81
코스, 한곳=>여수, 주가	 0.167	 0.9	 5.38
코스, 여수, 주가=>한곳	 0.167	 1.0	 4.31
한곳, 여수, 주가=>코스	 0.167	 1.0	 1.96
코스, 여수, 한곳=>주가	 0.167	 1.0	 5.81
한곳, 코스, 주가=>여수

전주, 국내=>맛집, 코스, 한곳	 0.157	 0.94	 5.47
코스, 국내=>전주, 맛집, 한곳	 0.157	 0.97	 5.32
한곳, 국내=>맛집, 전주, 코스	 0.157	 1.0	 5.08
전주, 맛집=>한곳, 코스, 국내	 0.157	 0.62	 3.97
코스, 맛집=>한곳, 전주, 국내	 0.157	 0.72	 4.61
맛집, 한곳=>전주, 코스, 국내	 0.157	 0.84	 5.18
전주, 코스=>한곳, 맛집, 국내	 0.157	 0.53	 3.38
전주, 한곳=>맛집, 코스, 국내	 0.157	 0.81	 5.02
코스, 한곳=>전주, 맛집, 국내	 0.157	 0.84	 5.05
전주, 맛집, 국내=>코스, 한곳	 0.157	 0.94	 5.05
코스, 맛집, 국내=>전주, 한곳	 0.157	 0.97	 5.02
한곳, 맛집, 국내=>전주, 코스	 0.157	 1.0	 3.38
전주, 코스, 국내=>맛집, 한곳	 0.157	 0.97	 5.18
한곳, 전주, 국내=>맛집, 코스	 0.157	 1.0	 4.61
한곳, 코스, 국내=>전주, 맛집	 0.157	 1.0	 3.97
코스, 전주, 맛집=>한곳, 국내	 0.157	 0.8	 5.08
전주, 맛집, 한곳=>코스, 국내	 0.157	 0.86	 5.32
코스, 맛집, 한곳=>전주, 국내	 0.157	 0.91	 5.47
전주, 코스, 한곳=>맛집, 국내	 0.157	 0.91	 5.47
코스, 전주, 맛집, 국내=>한곳	 0.157	 0.97	 4.18
한곳, 전주, 맛집, 국내=>코스	 0.157	 1.0	 1.96
한곳, 코스, 맛집, 국내=>전주	 0.157	 1.0	 2.0
한곳, 전주, 코스, 국내=>맛집	 0.157	 1.0	 3.26
코스, 전주, 맛집, 한곳=>국내	 0.157	 0.91	 5.31
국내=>한곳, 맛집, 코스, 주가	 0.157	 0.91	 5.31
맛집=>한곳, 코스, 국내, 주가	 0.157	 0.51	 3.26
주가=>한곳, 맛집, 코스, 국내	 0

전주, 여수, 한곳=>맛집, 주가	 0.167	 1.0	 5.81
한곳, 여수, 주가=>전주, 맛집	 0.167	 1.0	 3.97
한곳, 전주, 주가=>맛집, 여수	 0.167	 0.97	 5.81
전주, 맛집, 여수, 주가=>한곳	 0.167	 1.0	 4.31
전주, 맛집, 여수, 한곳=>주가	 0.167	 1.0	 5.81
한곳, 맛집, 여수, 주가=>전주	 0.167	 1.0	 2.0
한곳, 전주, 맛집, 주가=>여수	 0.167	 0.97	 5.64
한곳, 전주, 여수, 주가=>맛집	 0.167	 1.0	 3.26
맛집=>전주, 코스, 여수, 한곳	 0.167	 0.54	 3.26
여수=>맛집, 전주, 코스, 한곳	 0.167	 0.97	 5.64
전주=>맛집, 코스, 여수, 한곳	 0.167	 0.33	 2.0
코스=>전주, 맛집, 여수, 한곳	 0.167	 0.33	 1.96
한곳=>맛집, 전주, 코스, 여수	 0.167	 0.72	 4.31
맛집, 여수=>전주, 코스, 한곳	 0.167	 1.0	 5.81
전주, 맛집=>코스, 여수, 한곳	 0.167	 0.66	 3.97
코스, 맛집=>전주, 여수, 한곳	 0.167	 0.77	 4.61
맛집, 한곳=>전주, 코스, 여수	 0.167	 0.89	 5.35
전주, 여수=>맛집, 코스, 한곳	 0.167	 0.97	 5.64
코스, 여수=>전주, 맛집, 한곳	 0.167	 1.0	 5.49
여수, 한곳=>맛집, 전주, 코스	 0.167	 1.0	 5.08
전주, 코스=>맛집, 여수, 한곳	 0.167	 0.56	 3.38
전주, 한곳=>맛집, 코스, 여수	 0.167	 0.87	 5.18
코스, 한곳=>전주, 맛집, 여수	 0.167	 0.9	 5.38
전주, 맛집, 여수=>코스, 한곳	 0.167	 1.0	 5.38
코스, 맛집, 여수=>전주, 한곳	 0.167	 1.0	 5.18
맛집, 여수, 한곳=>전주, 코스	 0.167	 1.0	 3.38
코스, 전주, 맛집=>여수, 한곳	 0.167	 

주가=>전주, 제주도, 여수, 한곳	 0.152	 0.88	 5.81
한곳=>전주, 제주도, 여수, 주가	 0.152	 0.66	 4.31
전주, 여수=>한곳, 제주도, 주가	 0.152	 0.88	 5.81
제주도, 여수=>한곳, 전주, 주가	 0.152	 1.0	 5.81
여수, 주가=>전주, 제주도, 한곳	 0.152	 0.91	 5.99
여수, 한곳=>전주, 제주도, 주가	 0.152	 0.91	 5.99
전주, 제주도=>한곳, 여수, 주가	 0.152	 1.0	 5.99
전주, 주가=>제주도, 여수, 한곳	 0.152	 0.88	 5.81
전주, 한곳=>제주도, 여수, 주가	 0.152	 0.79	 5.18
제주도, 주가=>전주, 여수, 한곳	 0.152	 1.0	 5.99
제주도, 한곳=>전주, 여수, 주가	 0.152	 1.0	 5.99
한곳, 주가=>전주, 제주도, 여수	 0.152	 0.88	 5.81
전주, 제주도, 여수=>한곳, 주가	 0.152	 1.0	 5.81
전주, 여수, 주가=>제주도, 한곳	 0.152	 0.91	 5.99
전주, 여수, 한곳=>제주도, 주가	 0.152	 0.91	 5.99
제주도, 여수, 주가=>전주, 한곳	 0.152	 1.0	 5.18
제주도, 여수, 한곳=>전주, 주가	 0.152	 1.0	 5.81
한곳, 여수, 주가=>전주, 제주도	 0.152	 0.91	 5.99
전주, 제주도, 주가=>여수, 한곳	 0.152	 1.0	 5.99
전주, 제주도, 한곳=>여수, 주가	 0.152	 1.0	 5.99
한곳, 전주, 주가=>제주도, 여수	 0.152	 0.88	 5.81
한곳, 제주도, 주가=>전주, 여수	 0.152	 1.0	 5.81
전주, 제주도, 여수, 주가=>한곳	 0.152	 1.0	 4.31
전주, 제주도, 여수, 한곳=>주가	 0.152	 1.0	 5.81
한곳, 전주, 여수, 주가=>제주도	 0.152	 0.91	 5.99
한곳, 제주도, 여수, 주가=>전주	 0.152	 1.0	 2.0


맛집=>국내, 주가, 코스, 한곳, 여수	 0.157	 0.51	 3.26
여수=>국내, 주가, 코스, 한곳, 맛집	 0.157	 0.91	 5.81
주가=>국내, 코스, 한곳, 맛집, 여수	 0.157	 0.91	 5.81
코스=>국내, 주가, 한곳, 맛집, 여수	 0.157	 0.31	 1.96
한곳=>국내, 주가, 코스, 맛집, 여수	 0.157	 0.68	 4.31
맛집, 국내=>한곳, 코스, 여수, 주가	 0.157	 0.94	 5.63
여수, 국내=>한곳, 맛집, 코스, 주가	 0.157	 1.0	 5.81
주가, 국내=>맛집, 코스, 여수, 한곳	 0.157	 1.0	 5.99
코스, 국내=>한곳, 맛집, 여수, 주가	 0.157	 0.97	 5.8
한곳, 국내=>맛집, 코스, 여수, 주가	 0.157	 1.0	 5.99
맛집, 여수=>한곳, 코스, 국내, 주가	 0.157	 0.94	 5.99
맛집, 주가=>한곳, 코스, 여수, 국내	 0.157	 0.91	 5.81
코스, 맛집=>한곳, 국내, 여수, 주가	 0.157	 0.72	 4.61
맛집, 한곳=>코스, 국내, 여수, 주가	 0.157	 0.84	 5.35
여수, 주가=>한곳, 맛집, 코스, 국내	 0.157	 0.94	 5.99
코스, 여수=>한곳, 맛집, 국내, 주가	 0.157	 0.94	 5.99
여수, 한곳=>맛집, 코스, 국내, 주가	 0.157	 0.94	 5.99
코스, 주가=>한곳, 맛집, 여수, 국내	 0.157	 0.91	 5.81
한곳, 주가=>맛집, 코스, 여수, 국내	 0.157	 0.91	 5.81
코스, 한곳=>맛집, 국내, 여수, 주가	 0.157	 0.84	 5.38
맛집, 여수, 국내=>한곳, 코스, 주가	 0.157	 1.0	 5.81
맛집, 주가, 국내=>코스, 여수, 한곳	 0.157	 1.0	 5.99
코스, 맛집, 국내=>한곳, 여수, 주가	 0.157	 0.97	 5.8
한곳, 맛집, 국내=>코스, 여수, 주가	 0.157	 1.0	 5.99


전주, 여수, 한곳=>맛집, 코스, 제주도	 0.152	 0.91	 5.99
코스, 제주도, 여수=>전주, 맛집, 한곳	 0.152	 1.0	 5.49
제주도, 여수, 한곳=>맛집, 전주, 코스	 0.152	 1.0	 5.08
코스, 여수, 한곳=>맛집, 전주, 제주도	 0.152	 0.91	 5.99
코스, 전주, 제주도=>맛집, 여수, 한곳	 0.152	 1.0	 5.99
전주, 제주도, 한곳=>맛집, 코스, 여수	 0.152	 1.0	 5.99
전주, 코스, 한곳=>맛집, 제주도, 여수	 0.152	 0.88	 5.81
코스, 제주도, 한곳=>전주, 맛집, 여수	 0.152	 1.0	 5.99
전주, 맛집, 여수, 제주도=>코스, 한곳	 0.152	 1.0	 5.38
코스, 전주, 맛집, 여수=>제주도, 한곳	 0.152	 0.91	 5.99
전주, 맛집, 여수, 한곳=>코스, 제주도	 0.152	 0.91	 5.99
코스, 맛집, 여수, 제주도=>전주, 한곳	 0.152	 1.0	 5.18
한곳, 맛집, 여수, 제주도=>전주, 코스	 0.152	 1.0	 3.38
코스, 맛집, 여수, 한곳=>전주, 제주도	 0.152	 0.91	 5.99
코스, 전주, 맛집, 제주도=>여수, 한곳	 0.152	 1.0	 5.99
한곳, 전주, 맛집, 제주도=>코스, 여수	 0.152	 1.0	 5.99
코스, 전주, 맛집, 한곳=>제주도, 여수	 0.152	 0.88	 5.81
한곳, 코스, 맛집, 제주도=>전주, 여수	 0.152	 1.0	 5.81
코스, 전주, 제주도, 여수=>맛집, 한곳	 0.152	 1.0	 5.35
전주, 제주도, 여수, 한곳=>맛집, 코스	 0.152	 1.0	 4.61
전주, 코스, 여수, 한곳=>맛집, 제주도	 0.152	 0.91	 5.99
코스, 제주도, 여수, 한곳=>전주, 맛집	 0.152	 1.0	 3.97
코스, 전주, 제주도, 한곳=>맛집, 여수	 0.152	 1.0	 5.99
제주도, 코스, 전주, 맛집, 여수=>한곳	 0

맛집, 여수, 한곳=>전주, 코스, 국내, 주가	 0.157	 0.94	 5.99
전주, 맛집, 주가=>한곳, 코스, 여수, 국내	 0.157	 0.91	 5.81
코스, 전주, 맛집=>한곳, 국내, 여수, 주가	 0.157	 0.8	 5.08
전주, 맛집, 한곳=>코스, 국내, 여수, 주가	 0.157	 0.86	 5.49
코스, 맛집, 주가=>한곳, 전주, 여수, 국내	 0.157	 0.91	 5.81
한곳, 맛집, 주가=>전주, 코스, 여수, 국내	 0.157	 0.91	 5.81
코스, 맛집, 한곳=>전주, 국내, 여수, 주가	 0.157	 0.91	 5.81
전주, 여수, 주가=>한곳, 맛집, 코스, 국내	 0.157	 0.94	 5.99
전주, 코스, 여수=>한곳, 맛집, 국내, 주가	 0.157	 0.94	 5.99
전주, 여수, 한곳=>맛집, 코스, 국내, 주가	 0.157	 0.94	 5.99
코스, 여수, 주가=>한곳, 전주, 맛집, 국내	 0.157	 0.94	 5.99
한곳, 여수, 주가=>맛집, 전주, 코스, 국내	 0.157	 0.94	 5.8
코스, 여수, 한곳=>맛집, 전주, 국내, 주가	 0.157	 0.94	 5.99
전주, 코스, 주가=>한곳, 맛집, 여수, 국내	 0.157	 0.91	 5.81
한곳, 전주, 주가=>맛집, 코스, 여수, 국내	 0.157	 0.91	 5.81
전주, 코스, 한곳=>맛집, 국내, 여수, 주가	 0.157	 0.91	 5.81
한곳, 코스, 주가=>전주, 맛집, 여수, 국내	 0.157	 0.91	 5.81
전주, 맛집, 여수, 국내=>한곳, 코스, 주가	 0.157	 1.0	 5.81
주가, 맛집, 여수, 국내=>전주, 코스, 한곳	 0.157	 1.0	 5.81
코스, 맛집, 여수, 국내=>한곳, 전주, 주가	 0.157	 1.0	 5.81
한곳, 맛집, 여수, 국내=>전주, 코스, 주가	 0.157	 1.0	 5.81
전주, 맛집, 주가, 국내=>코스, 여수, 한곳	 0.157	 1.0	 

In [25]:
pd.options.display.max_rows

60

In [28]:
rules_df.head(60)

,lhs,rhs,지지도,신뢰도,향상도
0,제주도,"맛집, 여수",0.152,1.0,5.99
1,제주도,"여수, 주가",0.152,1.0,5.99
2,제주도,"코스, 여수",0.152,1.0,5.99
3,제주도,"여수, 한곳",0.152,1.0,5.99
4,"주가, 국내","맛집, 여수",0.157,1.0,5.99
5,"한곳, 국내","맛집, 여수",0.157,1.0,5.99
6,"주가, 국내","코스, 여수",0.157,1.0,5.99
7,"주가, 국내","여수, 한곳",0.157,1.0,5.99
8,"한곳, 국내","여수, 주가",0.157,1.0,5.99
9,"한곳, 국내","코스, 여수",0.157,1.0,5.99
